<a href="https://colab.research.google.com/github/Ali478/-CNN-for-MNIST-digit-recognition./blob/main/CNN_for_MNIST_digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

In [3]:
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)


In [4]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/train-labels-idx1-ubyte.gz to /files/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-images-idx3-ubyte.gz to /files/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /files/MNIST/raw/t10k-labels-idx1-ubyte.gz to /files/MNIST/raw



In [5]:
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [6]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

example_data.shape

torch.Size([1000, 1, 28, 28])

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)

In [8]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [9]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [10]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
    #  torch.save(network.state_dict(), '/results/model.pth')
     # torch.save(optimizer.state_dict(), '/results/optimizer.pth')


In [11]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


In [12]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3096, Accuracy: 924/10000 (9%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311163
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.279947
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.274196
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.242758
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.186769
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.156291
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.972376
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.859842
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.506459
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.161100
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.952828
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.803664
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.754260
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.637047
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.516859
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.521194
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.567992
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.676831
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.608502
Train Epoch: 1 [12160/6

In [13]:
with torch.no_grad():
  output = network(example_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [14]:
continued_network = Net()
continued_optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                                momentum=momentum)

In [15]:
for i in range(4,10):
  test_counter.append(i*len(train_loader.dataset))
  train(i)
  test()


Train Epoch: 4 [0/60000 (0%)]	Loss: 0.014510


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Train Epoch: 4 [640/60000 (1%)]	Loss: 0.016676
Train Epoch: 4 [1280/60000 (2%)]	Loss: 0.064343
Train Epoch: 4 [1920/60000 (3%)]	Loss: 0.015373
Train Epoch: 4 [2560/60000 (4%)]	Loss: 0.029759
Train Epoch: 4 [3200/60000 (5%)]	Loss: 0.040784
Train Epoch: 4 [3840/60000 (6%)]	Loss: 0.049585
Train Epoch: 4 [4480/60000 (7%)]	Loss: 0.097242
Train Epoch: 4 [5120/60000 (9%)]	Loss: 0.056676
Train Epoch: 4 [5760/60000 (10%)]	Loss: 0.043417
Train Epoch: 4 [6400/60000 (11%)]	Loss: 0.116105
Train Epoch: 4 [7040/60000 (12%)]	Loss: 0.081111
Train Epoch: 4 [7680/60000 (13%)]	Loss: 0.100602
Train Epoch: 4 [8320/60000 (14%)]	Loss: 0.081303
Train Epoch: 4 [8960/60000 (15%)]	Loss: 0.081862
Train Epoch: 4 [9600/60000 (16%)]	Loss: 0.117012
Train Epoch: 4 [10240/60000 (17%)]	Loss: 0.245640
Train Epoch: 4 [10880/60000 (18%)]	Loss: 0.077372
Train Epoch: 4 [11520/60000 (19%)]	Loss: 0.061239
Train Epoch: 4 [12160/60000 (20%)]	Loss: 0.093932
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.092551
Train Epoch: 4 [13440/60

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.0512, Accuracy: 9830/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.080048
Train Epoch: 5 [640/60000 (1%)]	Loss: 0.091759
Train Epoch: 5 [1280/60000 (2%)]	Loss: 0.113282
Train Epoch: 5 [1920/60000 (3%)]	Loss: 0.011007
Train Epoch: 5 [2560/60000 (4%)]	Loss: 0.058987
Train Epoch: 5 [3200/60000 (5%)]	Loss: 0.016120
Train Epoch: 5 [3840/60000 (6%)]	Loss: 0.156100
Train Epoch: 5 [4480/60000 (7%)]	Loss: 0.009859
Train Epoch: 5 [5120/60000 (9%)]	Loss: 0.091237
Train Epoch: 5 [5760/60000 (10%)]	Loss: 0.034650
Train Epoch: 5 [6400/60000 (11%)]	Loss: 0.084375
Train Epoch: 5 [7040/60000 (12%)]	Loss: 0.050592
Train Epoch: 5 [7680/60000 (13%)]	Loss: 0.205673
Train Epoch: 5 [8320/60000 (14%)]	Loss: 0.103720
Train Epoch: 5 [8960/60000 (15%)]	Loss: 0.037490
Train Epoch: 5 [9600/60000 (16%)]	Loss: 0.046590
Train Epoch: 5 [10240/60000 (17%)]	Loss: 0.073506
Train Epoch: 5 [10880/60000 (18%)]	Loss: 0.009812
Train Epoch: 5 [11520/60000 (19%)]	Loss: 0.028288
Train Epoch: 5 [12160